## GAN 50 sqr

This notebook is modified from the keras repository: https://github.com/BAI-Yeqi/Keras-GAN/blob/master/gan/gan.py

As tested (256, 256) is too big a input, so we are using (50, 50)


## The generator is only getting worse, need to weaken the discriminator


In [4]:
from __future__ import print_function, division
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers import  MaxPooling2D
from keras import backend as K
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.models import load_model

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [7]:
X_try = np.load("drive/Colab Notebooks/Data/ChestX_8/50sqr/image_array_batch_1.npy")
X_try.shape

(2000, 50, 50)

The Training Set of MNIST got the shape 

```
(60000, 28, 28)
```



In [8]:
class GAN():
    def __init__(self):
        self.img_rows = 50 
        self.img_cols = 50
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
            optimizer=optimizer,
            metrics=['accuracy'])
        self.discriminator.summary()

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)
        self.generator.summary()
        
        # The generator takes noise as input and generated imgs
        z = Input(shape=(400,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        return load_model("drive/Colab Notebooks/GAN Medical Image/Models/Fir_GAN_G_epoch_15000.h5")

    def build_discriminator(self):
        
        return load_model("drive/Colab Notebooks/GAN Medical Image/Models/D_CNN_CIFAR.h5")

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = np.load("drive/Colab Notebooks/Data/ChestX_8/50sqr/image_array_batch_1.npy")
        
        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3) # to resize the images array into size (2000, 50, 50, 1)
        print("X_train shape: " + str(X_train.shape))

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch) 
            # idx is an array of length = half_batch and value in range of the no. of training samples
            imgs = X_train[idx] # imgs are then selected from the X_train

            noise = np.random.normal(0, 1, (half_batch, 400))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise) # shape: (half_batch, 50, 50)

            # Train the discriminator (and the loss will be returned)
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 400))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones) (128 numbers of 1s contained in the 1 d array)
            valid_y = np.array([1] * batch_size)
            
            
            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
                
            if epoch % 5000 == 0:
                self.generator.save("drive/Colab Notebooks/GAN Medical Image/Models/Stage2/G-G15000_CNN_epoch_"+ str(epoch) +".h5")
                self.discriminator.save("drive/Colab Notebooks/GAN Medical Image/Models/Stage2/D-G15000_CNN_epoch_"+ str(epoch) +".h5")

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 400))
        print("shape of noise: "+ str(noise.shape))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("drive/Colab Notebooks/GAN Medical Image/Images Gen/Image GAN Stage2/G-G15000__D-CNN_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)

/usr/local/lib/python3.6/dist-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 44, 44, 3)         150       
_________________________________________________________________
activation_10 (Activation)   (None, 44, 44, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 38, 38, 3)         444       
_________________________________________________________________
activation_11 (Activation)   (None, 38, 38, 3)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 3)         444       
_________________________________________________________________
activation_12 (Activation)   (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1_0 (Conv2D)          (None, 32, 32, 32)        896       
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.718782, acc.: 50.00%] [G loss: 0.851148]
shape of noise: (25, 400)
1 [D loss: 0.707800, acc.: 50.00%] [G loss: 0.845354]
2 [D loss: 0.701284, acc.: 50.00%] [G loss: 0.840924]
3 [D loss: 0.708772, acc.: 50.00%] [G loss: 0.841238]
4 [D loss: 0.707779, acc.: 50.00%] [G loss: 0.832312]
5 [D loss: 0.705974, acc.: 50.00%] [G loss: 0.829620]
6 [D loss: 0.709204, acc.: 50.00%] [G loss: 0.830079]
7 [D loss: 0.703958, acc.: 50.00%] [G loss: 0.828614]
8 [D loss: 0.702057, acc.: 50.00%] [G loss: 0.824281]
9 [D loss: 0.705593, acc.: 50.00%] [G loss: 0.814155]
10 [D loss: 0.705984, acc.: 50.00%] [G loss: 0.819276]
11 [D loss: 0.700911, acc.: 50.00%] [G loss: 0.815917]
12 [D loss: 0.703779, acc.: 50.00%] [G loss: 0.806050]
13 [D loss: 0.705655, acc.: 50.00%] [G loss: 0.804545]
14 [D loss: 0.703147, acc.: 50.00%] [G loss: 0.802214]
15 [D loss: 0.702725, acc.: 50.00%] [G loss: 0.800480]
16 [D loss: 0.702964, acc.: 50.00%] [G loss: 0.794359]
17 [D loss: 0.706098, acc.: 50.00%] [G loss: 0.78

101 [D loss: 0.689261, acc.: 43.75%] [G loss: 0.712060]
102 [D loss: 0.694947, acc.: 53.12%] [G loss: 0.755297]
103 [D loss: 0.685480, acc.: 50.00%] [G loss: 0.721918]
104 [D loss: 0.692732, acc.: 53.12%] [G loss: 0.714775]
105 [D loss: 0.684517, acc.: 53.12%] [G loss: 0.788390]
106 [D loss: 0.691746, acc.: 46.88%] [G loss: 0.720398]
107 [D loss: 0.687934, acc.: 46.88%] [G loss: 0.725752]
108 [D loss: 0.687510, acc.: 43.75%] [G loss: 0.715261]
109 [D loss: 0.688433, acc.: 50.00%] [G loss: 0.722422]
110 [D loss: 0.692534, acc.: 46.88%] [G loss: 0.712930]
111 [D loss: 0.690560, acc.: 75.00%] [G loss: 0.714822]
112 [D loss: 0.685460, acc.: 59.38%] [G loss: 0.718633]
113 [D loss: 0.667538, acc.: 65.62%] [G loss: 0.711467]
114 [D loss: 0.685553, acc.: 31.25%] [G loss: 0.702802]
115 [D loss: 0.694624, acc.: 46.88%] [G loss: 0.745105]
116 [D loss: 0.679998, acc.: 43.75%] [G loss: 0.727491]
117 [D loss: 0.655308, acc.: 71.88%] [G loss: 0.975089]
118 [D loss: 0.685986, acc.: 43.75%] [G loss: 0.

...

2960 [D loss: 0.218352, acc.: 100.00%] [G loss: 1.596130]
2961 [D loss: 0.218302, acc.: 100.00%] [G loss: 1.596341]
2962 [D loss: 0.218252, acc.: 100.00%] [G loss: 1.596553]
2963 [D loss: 0.218202, acc.: 100.00%] [G loss: 1.596764]
2964 [D loss: 0.218152, acc.: 100.00%] [G loss: 1.596975]
2965 [D loss: 0.218102, acc.: 100.00%] [G loss: 1.597187]
2966 [D loss: 0.218053, acc.: 100.00%] [G loss: 1.597399]
2967 [D loss: 0.218003, acc.: 100.00%] [G loss: 1.597610]
2968 [D loss: 0.217953, acc.: 100.00%] [G loss: 1.597821]
2969 [D loss: 0.217903, acc.: 100.00%] [G loss: 1.598033]
2970 [D loss: 0.217854, acc.: 100.00%] [G loss: 1.598244]
2971 [D loss: 0.217803, acc.: 100.00%] [G loss: 1.598456]
2972 [D loss: 0.217753, acc.: 100.00%] [G loss: 1.598667]
2973 [D loss: 0.217704, acc.: 100.00%] [G loss: 1.598879]
2974 [D loss: 0.217654, acc.: 100.00%] [G loss: 1.599090]
2975 [D loss: 0.217604, acc.: 100.00%] [G loss: 1.599302]
2976 [D loss: 0.217554, acc.: 100.00%] [G loss: 1.599513]
2977 [D loss: 

3058 [D loss: 0.213510, acc.: 100.00%] [G loss: 1.616878]
3059 [D loss: 0.213462, acc.: 100.00%] [G loss: 1.617090]
3060 [D loss: 0.213413, acc.: 100.00%] [G loss: 1.617302]
3061 [D loss: 0.213364, acc.: 100.00%] [G loss: 1.617514]
3062 [D loss: 0.213315, acc.: 100.00%] [G loss: 1.617726]
3063 [D loss: 0.213266, acc.: 100.00%] [G loss: 1.617938]
3064 [D loss: 0.213218, acc.: 100.00%] [G loss: 1.618150]
3065 [D loss: 0.213169, acc.: 100.00%] [G loss: 1.618362]
3066 [D loss: 0.213120, acc.: 100.00%] [G loss: 1.618574]
3067 [D loss: 0.213071, acc.: 100.00%] [G loss: 1.618786]
3068 [D loss: 0.213022, acc.: 100.00%] [G loss: 1.618998]
3069 [D loss: 0.212974, acc.: 100.00%] [G loss: 1.619210]
3070 [D loss: 0.212925, acc.: 100.00%] [G loss: 1.619423]
3071 [D loss: 0.212876, acc.: 100.00%] [G loss: 1.619635]
3072 [D loss: 0.212827, acc.: 100.00%] [G loss: 1.619847]
3073 [D loss: 0.212779, acc.: 100.00%] [G loss: 1.620059]
3074 [D loss: 0.212730, acc.: 100.00%] [G loss: 1.620271]
3075 [D loss: 

3157 [D loss: 0.208723, acc.: 100.00%] [G loss: 1.637905]
3158 [D loss: 0.208675, acc.: 100.00%] [G loss: 1.638118]
3159 [D loss: 0.208627, acc.: 100.00%] [G loss: 1.638331]
3160 [D loss: 0.208579, acc.: 100.00%] [G loss: 1.638543]
3161 [D loss: 0.208532, acc.: 100.00%] [G loss: 1.638756]
3162 [D loss: 0.208484, acc.: 100.00%] [G loss: 1.638969]
3163 [D loss: 0.208436, acc.: 100.00%] [G loss: 1.639182]
3164 [D loss: 0.208388, acc.: 100.00%] [G loss: 1.639394]
3165 [D loss: 0.208341, acc.: 100.00%] [G loss: 1.639607]
3166 [D loss: 0.208293, acc.: 100.00%] [G loss: 1.639820]
3167 [D loss: 0.208245, acc.: 100.00%] [G loss: 1.640033]
3168 [D loss: 0.208197, acc.: 100.00%] [G loss: 1.640246]
3169 [D loss: 0.208150, acc.: 100.00%] [G loss: 1.640458]
3170 [D loss: 0.208102, acc.: 100.00%] [G loss: 1.640671]
3171 [D loss: 0.208054, acc.: 100.00%] [G loss: 1.640884]
3172 [D loss: 0.208006, acc.: 100.00%] [G loss: 1.641097]
3173 [D loss: 0.207959, acc.: 100.00%] [G loss: 1.641310]
3174 [D loss: 

3253 [D loss: 0.204178, acc.: 100.00%] [G loss: 1.658360]
3254 [D loss: 0.204131, acc.: 100.00%] [G loss: 1.658573]
3255 [D loss: 0.204084, acc.: 100.00%] [G loss: 1.658787]
3256 [D loss: 0.204037, acc.: 100.00%] [G loss: 1.659000]
3257 [D loss: 0.203990, acc.: 100.00%] [G loss: 1.659214]
3258 [D loss: 0.203943, acc.: 100.00%] [G loss: 1.659427]
3259 [D loss: 0.203897, acc.: 100.00%] [G loss: 1.659641]
3260 [D loss: 0.203850, acc.: 100.00%] [G loss: 1.659854]
3261 [D loss: 0.203803, acc.: 100.00%] [G loss: 1.660068]
3262 [D loss: 0.203756, acc.: 100.00%] [G loss: 1.660281]
3263 [D loss: 0.203709, acc.: 100.00%] [G loss: 1.660495]
3264 [D loss: 0.203663, acc.: 100.00%] [G loss: 1.660708]
3265 [D loss: 0.203616, acc.: 100.00%] [G loss: 1.660922]
3266 [D loss: 0.203569, acc.: 100.00%] [G loss: 1.661136]
3267 [D loss: 0.203523, acc.: 100.00%] [G loss: 1.661349]
3268 [D loss: 0.203476, acc.: 100.00%] [G loss: 1.661563]
3269 [D loss: 0.203429, acc.: 100.00%] [G loss: 1.661776]
3270 [D loss: 

KeyboardInterrupt: ignored

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install -q keras

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 841117307224395232, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11297803469
 locality {
   bus_id: 1
 }
 incarnation: 6319703760253393801
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]